# Wyszukiwarka semantyczna wykorzystująca SVD

### Autor: Jakub Psarski

## 1. Wprowadzenie

Projekt zawiera implementację wyszukiwarki tekstowej z wykorzystaniem dekompozycji według wartości osobliwych (Singular Value Decomposition/SVD). Celem projektu było stworzenie efektywnego systemu wyszukiwania dokumentów, z możliwością porównania tradycyjnej metody opartej na podobieństwie cosinusowym wektorów TF-IDF oraz metody wykorzystującej redukcję wymiarowości poprzez SVD (znaną również jako Latent Semantic Indexing/LSI).

## 2. Struktura projektu

Projekt został podzielony na następujące moduły:

1. **Moduł przetwarzania tekstu** (`src/preprocessing.py`):
   - Budowanie słownika terminów
   - Tworzenie macierzy term-document
   - Implementacja transformacji IDF

2. **Moduł SVD** (`src/svd.py`):
   - Implementacja dekompozycji według wartości osobliwych
   - Redukcja wymiarowości macierzy

3. **Moduł wyszukiwania** (`src/search.py`):
   - Konwersja zapytań na wektory
   - Implementacja wyszukiwania bez SVD
   - Implementacja wyszukiwania z wykorzystaniem SVD

4. **Moduł konfiguracyjny** (`src/setup.py`):
   - Konfiguracja i inicjalizacja komponentów wyszukiwarki
   - Zarządzanie zapisem i odczytem obliczonych danych

5. **Interfejs konsolowy** (`main.py`):
   - Obsługa poleceń użytkownika
   - Wyświetlanie wyników wyszukiwania

6. **Interfejs webowy** (`app.py` oraz katalog `client`):
   - REST API w Flasku
   - Interfejs użytkownika w HTML/CSS/JavaScript

7. **Web crawler** (`src/crawler`):
    - Crawler indeksujący artykuły Wikipedii (Simple English)
    - Przygotowanie bazy danych SQLite zawierającej zaindeksowane artykuły

## 3. Realizacja głównych elementów zadania

### 3.1 Budowa słownika i macierzy term-document

Proces budowy słownika zaimplementowano poprzez funkcję `build_vocabulary()` w `src/preprocessing.py`:

- **Eliminacja stop-words**: Wykorzystano bibliotekę NLTK do usunięcia słów nieznaczących (np. "the", "and", "of").

- **Filtrowanie rzadkich terminów**: Zastosowano parametr `min_df` (domyślnie 5), eliminujący terminy występujące w mniej niż określonej liczbie dokumentów. Poprawia to jakość wyszukiwania, usuwając szum i pozostawiając jedynie terminy, które wprowadzają znaczenie semantyczne.

- **Filtrowanie powszechnych terminów**: Zastosowano parametr `max_df` (domyślnie 0.5), eliminujący terminy występujące w więcej niż określonym procencie dokumentów, co stanowi uzupełnienie do eliminacji stop-words.

Następnie zaimplementowano tworzenie macierzy term-document w funkcji `create_document_vectors()`:
- Wykorzystano format macierzy rzadkiej CSR z biblioteki SciPy

### 3.2 Transformacja IDF i normalizacja

W funkcji `apply_idf()` zaimplementowano transformację TF-IDF:
- Obliczanie wartości IDF zgodnie z formułą: `IDF(term) = log(N / df(term))`
- Zastosowanie tych wag do macierzy term-document

Dodatkowo zastosowano normalizację wektorów dokumentów, co jest kluczowe dla późniejszego obliczania podobieństwa cosinusowego.

### 3.3 Implementacja SVD

Zgodnie z wymaganiami, zaimplementowano redukcję wymiarowości przy użyciu SVD w pliku `src/svd.py`:
- Wykorzystanie funkcji `svds` z biblioteki SciPy
- Umożliwienie wyboru parametru k (wymiaru macierzy po redukcji)
- Zapisywanie wyników SVD dla różnych wartości k w celu przyspieszenia pracy

### 3.4 Funkcje wyszukiwania

Zaimplementowano dwie funkcje wyszukiwania:

1. **Wyszukiwanie tradycyjne** (`search_documents()`):
   - Konwersja zapytania na wektor w przestrzeni terminów
   - Obliczanie podobieństwa cosinusowego między zapytaniem a dokumentami
   - Sortowanie wyników według podobieństwa

2. **Wyszukiwanie z SVD** (`search_documents_svd()`):
   - Konwersja zapytania na wektor w przestrzeni terminów
   - Projekcja zapytania do przestrzeni pojęć (konceptów) poprzez macierz U
   - Obliczanie podobieństwa w zredukowanej przestrzeni
   - Sortowanie wyników według podobieństwa

## 4. Eksperymenty i analiza wyników

Przeprowadzono szereg eksperymentów porównujących:
- Wyszukiwanie z IDF i bez IDF
- Wyszukiwanie bez SVD oraz z SVD dla różnych wartości k (100, 200, 500, 1000, 2000)

### 4.1. Importy i konfiguracja

In [1]:
import sqlite3
import time

from IPython.display import Markdown, display
from sklearn.preprocessing import normalize
from src.preprocessing import apply_idf
from src.search import search_documents, search_documents_svd
from src.setup import (load_or_compute_svd, load_or_compute_term_doc_matrix,
                       load_or_compute_vocabulary)

DB_PATH = './data/simplified_wiki_index.db'

### 4.2. Budowanie słownika i macierzy term-document, IDF, normalizacja

In [2]:
vocabulary = load_or_compute_vocabulary()

term_doc_matrix, doc_ids = load_or_compute_term_doc_matrix(vocabulary)

print("\nApplying IDF transformation...")
start_time = time.time()
term_doc_matrix_idf, idf_values = apply_idf(term_doc_matrix)
print(f"TIME: {time.time() - start_time:.2f}s")

print("\nNormalizing matrices...")
start_time = time.time()
normalized_term_doc_matrix = normalize(term_doc_matrix, axis=1)
normalized_term_doc_matrix_idf = normalize(term_doc_matrix_idf, axis=1)
print(f"TIME: {time.time() - start_time:.2f}s")


Loading precomputed vocabulary from file...

Loading precomputed term-document matrix and document IDs from file...

Applying IDF transformation...
TIME: 0.21s

Normalizing matrices...
TIME: 0.13s


### 4.3. Porównanie wyników wyszukiwania dla różnych konfiguracji

In [4]:
test_queries = [
    "artificial intelligence",
    "quantum physics",
    "history of Poland",
    "climate change",
    "machine learning algorithms"
]

k_values = [100, 200, 500, 1000]
results_table = {}

for query in test_queries:
    results_table[query] = {}
    results = search_documents(query, normalized_term_doc_matrix_idf, vocabulary, doc_ids, idf_values)
    results_table[query]["no_svd_idf"] = results

for k_svd in k_values:
    svd_components = load_or_compute_svd(True, normalized_term_doc_matrix_idf, k_svd)
    for query in test_queries:
        results = search_documents_svd(query, svd_components, vocabulary, doc_ids, idf_values)
        results_table[query][f"svd_k={k_svd}_idf"] = results

for query in test_queries:
    results = search_documents(query, normalized_term_doc_matrix, vocabulary, doc_ids, None)
    results_table[query]["no_svd_no_idf"] = results

for k_svd in k_values:
    svd_components = load_or_compute_svd(False, normalized_term_doc_matrix, k_svd)
    for query in test_queries:
        results = search_documents_svd(query, svd_components, vocabulary, doc_ids, None)
        results_table[query][f"svd_k={k_svd}_no_idf"] = results

all_doc_ids = set()
for query in test_queries:
    for config_results in results_table[query].values():
        all_doc_ids.update([doc_id for doc_id, _ in config_results])

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
titles = {}
for doc_id in all_doc_ids:
    cursor.execute("SELECT title FROM pages WHERE id = ?", (doc_id,))
    title = cursor.fetchone()[0]
    titles[doc_id] = title
conn.close()

for query in test_queries:
    markdown_table = f"## Zapytanie: '{query}'\n\n"
    configs = ["bez SVD"] + [f"SVD k={k}" for k in k_values]
    markdown_table += f"|     | {' | '.join(configs)} |\n"
    markdown_table += f"| --- | {' | '.join(['---' for _ in configs])} |\n"

    row = ["**z IDF**"]

    for config in ["no_svd_idf"] + [f"svd_k={k}_idf" for k in k_values]:
        results = results_table[query][config] if results_table[query][config] else []
        cell_content = []

        for i, (doc_id, score) in enumerate(results):
            doc_title = titles[doc_id]
            if len(doc_title) > 30:
                doc_title = doc_title[:27] + "..."
            cell_content.append(f"{i+1}. ({score:.4f}) {doc_title}")

        row.append("<br>".join(cell_content) if cell_content else "---")

    markdown_table += f"| {' | '.join(row)} |\n"

    row = ["**bez IDF**"]

    for config in ["no_svd_no_idf"] + [f"svd_k={k}_no_idf" for k in k_values]:
        results = results_table[query][config] if results_table[query][config] else []
        cell_content = []

        for i, (doc_id, score) in enumerate(results):
            doc_title = titles[doc_id]
            if len(doc_title) > 30:
                doc_title = doc_title[:27] + "..."
            cell_content.append(f"{i+1}. ({score:.4f}) {doc_title}")

        row.append("<br>".join(cell_content) if cell_content else "---")

    markdown_table += f"| {' | '.join(row)} |\n"

    display(Markdown(markdown_table))


Loading precomputed SVD components (k=100, idf) from file...

Loading precomputed SVD components (k=200, idf) from file...

Loading precomputed SVD components (k=500, idf) from file...

Loading precomputed SVD components (k=1000, idf) from file...

Loading precomputed SVD components (k=100, no idf) from file...

Loading precomputed SVD components (k=200, no idf) from file...

Loading precomputed SVD components (k=500, no idf) from file...

Loading precomputed SVD components (k=1000, no idf) from file...


## Zapytanie: 'artificial intelligence'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.4471) Artificial intelligence<br>2. (0.2390) Intelligence quotient<br>3. (0.1954) Intelligence<br>4. (0.1874) Military intelligence<br>5. (0.1792) Anne Anastasi<br>6. (0.1676) Evolution<br>7. (0.1603) Isabelle Ryl<br>8. (0.1508) Artificial<br>9. (0.1365) Joint Intelligence Committee<br>10. (0.1265) Blog | 1. (0.7470) Human biology<br>2. (0.7414) Trait<br>3. (0.7390) Darwin–Wallace Medal<br>4. (0.7373) E. B. Ford<br>5. (0.7357) Macroevolution<br>6. (0.7349) Systematics<br>7. (0.7325) Modern evolutionary synthesis<br>8. (0.7315) Darwinism<br>9. (0.7307) Evolutionary algorithm<br>10. (0.7300) Thomas Hunt Morgan | 1. (0.7363) Evolutionary algorithm<br>2. (0.7325) Human biology<br>3. (0.7287) Theodosius Dobzhansky<br>4. (0.7266) Darwinism<br>5. (0.7219) Lamarckism<br>6. (0.7196) Evolutionary biology<br>7. (0.7149) Social darwinism<br>8. (0.7131) Modern evolutionary synthesis<br>9. (0.7122) Evolution<br>10. (0.7108) Ernst Mayr | 1. (0.6570) List of artificial islands<br>2. (0.5933) Global Artificial Intellige...<br>3. (0.5277) Artificial intelligence<br>4. (0.5172) Evolution<br>5. (0.5171) Isabelle Ryl<br>6. (0.5160) Social responses to the ide...<br>7. (0.5112) Sewall's Point, Florida<br>8. (0.5058) Realization<br>9. (0.5051) Macroevolution<br>10. (0.4974) Human biology | 1. (0.6368) List of artificial islands<br>2. (0.6080) Global Artificial Intellige...<br>3. (0.5753) Artificial intelligence<br>4. (0.5618) Isabelle Ryl<br>5. (0.5293) Artificial<br>6. (0.5220) Artificial language<br>7. (0.4629) Intelligence quotient<br>8. (0.4413) Spearman, Texas<br>9. (0.4393) Intelligence agency<br>10. (0.4330) John McCarthy |
| **bez IDF** | 1. (0.4438) Artificial intelligence<br>2. (0.2471) Intelligence quotient<br>3. (0.1988) Intelligence<br>4. (0.1928) Military intelligence<br>5. (0.1854) Anne Anastasi<br>6. (0.1624) Evolution<br>7. (0.1592) Isabelle Ryl<br>8. (0.1462) Artificial<br>9. (0.1412) Joint Intelligence Committee<br>10. (0.1253) Blog | 1. (0.7359) Human biology<br>2. (0.7295) Trait<br>3. (0.7285) Darwin–Wallace Medal<br>4. (0.7254) E. B. Ford<br>5. (0.7242) Systematics<br>6. (0.7241) Macroevolution<br>7. (0.7210) Global Artificial Intellige...<br>8. (0.7209) Modern evolutionary synthesis<br>9. (0.7196) Autopoiesis<br>10. (0.7196) Evolutionary algorithm | 1. (0.7252) Evolutionary algorithm<br>2. (0.7249) Human biology<br>3. (0.7169) Theodosius Dobzhansky<br>4. (0.7129) Darwinism<br>5. (0.7109) Evolutionary biology<br>6. (0.7096) Lamarckism<br>7. (0.7028) Social darwinism<br>8. (0.7016) Modern evolutionary synthesis<br>9. (0.7004) Ernst Mayr<br>10. (0.6987) Evolution | 1. (0.6419) List of artificial islands<br>2. (0.5907) Global Artificial Intellige...<br>3. (0.5318) Artificial intelligence<br>4. (0.5173) Isabelle Ryl<br>5. (0.5027) Social responses to the ide...<br>6. (0.5022) Evolution<br>7. (0.4992) Sewall's Point, Florida<br>8. (0.4937) Human biology<br>9. (0.4926) Macroevolution<br>10. (0.4910) Realization | 1. (0.6222) List of artificial islands<br>2. (0.6040) Global Artificial Intellige...<br>3. (0.5764) Artificial intelligence<br>4. (0.5604) Isabelle Ryl<br>5. (0.5169) Artificial<br>6. (0.5107) Artificial language<br>7. (0.4736) Intelligence quotient<br>8. (0.4529) Spearman, Texas<br>9. (0.4505) Intelligence agency<br>10. (0.4425) Hansford County, Texas |


## Zapytanie: 'quantum physics'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.5724) Quantum mechanics<br>2. (0.4702) Physics<br>3. (0.3427) String theory<br>4. (0.2412) Uncertainty principle<br>5. (0.2371) Quantum biology<br>6. (0.2364) Branches of physics<br>7. (0.2201) Quantum computer<br>8. (0.2176) Quantum entanglement<br>9. (0.1657) Albert Einstein<br>10. (0.1287) Atomic physics | 1. (0.9961) List of physicists<br>2. (0.9891) Dirac fermion<br>3. (0.9888) Matter wave<br>4. (0.9884) Boson<br>5. (0.9851) Quantum<br>6. (0.9849) Ghost (physics)<br>7. (0.9844) Quantum chromodynamics<br>8. (0.9831) Quantum mechanics<br>9. (0.9831) Quantum electrodynamics<br>10. (0.9830) Paul Dirac | 1. (0.9670) Paul Dirac<br>2. (0.9439) Skyrmion<br>3. (0.9394) Dirac equation<br>4. (0.9363) Old quantum theory<br>5. (0.9301) Photoelectric effect<br>6. (0.9287) Photon<br>7. (0.9285) Classical physics<br>8. (0.9260) Quantum theory<br>9. (0.9257) Ghost (physics)<br>10. (0.9241) Pauli exclusion principle | 1. (0.8615) List of physicists<br>2. (0.8563) Classical physics<br>3. (0.8549) Paul Dirac<br>4. (0.8435) Quantum gravity<br>5. (0.8314) Wave–particle duality<br>6. (0.8217) Skyrmion<br>7. (0.8148) Ghost (physics)<br>8. (0.8083) Quantum entanglement<br>9. (0.8017) Quantum theory<br>10. (0.7999) Interpretations of quantum ... | 1. (0.8210) Classical physics<br>2. (0.8165) Quantum mechanics<br>3. (0.7885) Quantum theory<br>4. (0.7880) Quantum gravity<br>5. (0.7695) Wave–particle duality<br>6. (0.7563) Copenhagen interpretation<br>7. (0.7484) Niels Bohr<br>8. (0.7425) Paul Dirac<br>9. (0.7238) Young's double-slit experiment<br>10. (0.7137) Old quantum theory |
| **bez IDF** | 1. (0.5423) Quantum mechanics<br>2. (0.5089) Physics<br>3. (0.3210) String theory<br>4. (0.2548) Branches of physics<br>5. (0.2303) Uncertainty principle<br>6. (0.2238) Quantum biology<br>7. (0.2025) Quantum computer<br>8. (0.2017) Quantum entanglement<br>9. (0.1691) Albert Einstein<br>10. (0.1413) Atomic physics | 1. (0.9965) List of physicists<br>2. (0.9891) Dirac fermion<br>3. (0.9886) Matter wave<br>4. (0.9881) Boson<br>5. (0.9852) Quantum<br>6. (0.9849) Ghost (physics)<br>7. (0.9848) Quantum chromodynamics<br>8. (0.9836) Quantum electrodynamics<br>9. (0.9835) Paul Dirac<br>10. (0.9825) Quantum mechanics | 1. (0.9667) Paul Dirac<br>2. (0.9455) Skyrmion<br>3. (0.9433) Dirac equation<br>4. (0.9338) Old quantum theory<br>5. (0.9328) Ghost (physics)<br>6. (0.9309) Classical physics<br>7. (0.9272) Quantum theory<br>8. (0.9271) Photoelectric effect<br>9. (0.9255) Bose-Einstein statistics<br>10. (0.9239) Photon | 1. (0.8664) List of physicists<br>2. (0.8654) Classical physics<br>3. (0.8620) Paul Dirac<br>4. (0.8502) Quantum gravity<br>5. (0.8255) Wave–particle duality<br>6. (0.8225) Ghost (physics)<br>7. (0.8208) Skyrmion<br>8. (0.8023) Physicist<br>9. (0.7973) Quantum entanglement<br>10. (0.7896) Quantum theory | 1. (0.8368) Classical physics<br>2. (0.8018) Quantum gravity<br>3. (0.7921) Quantum mechanics<br>4. (0.7666) Quantum theory<br>5. (0.7576) Wave–particle duality<br>6. (0.7402) Niels Bohr<br>7. (0.7385) Copenhagen interpretation<br>8. (0.7384) Paul Dirac<br>9. (0.7077) Young's double-slit experiment<br>10. (0.7032) Old quantum theory |


## Zapytanie: 'history of Poland'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.4226) Poland<br>2. (0.1663) History of Poland<br>3. (0.1650) Józef Piłsudski<br>4. (0.1345) Soviet persecution of Poles...<br>5. (0.1219) Auschwitz concentration camp<br>6. (0.1142) Prussia<br>7. (0.1105) Double genocide theory<br>8. (0.1053) Białystok<br>9. (0.1032) Historiography<br>10. (0.1028) World War II | 1. (0.7336) History of Poland<br>2. (0.7173) Skierniewice<br>3. (0.7100) Slovenia<br>4. (0.7100) Poland<br>5. (0.7057) Malbork<br>6. (0.7057) Kwidzyn<br>7. (0.7056) Soviet invasion of Poland<br>8. (0.7055) Volhynia<br>9. (0.7037) Micula (surname)<br>10. (0.6980) Pińczów | 1. (0.8000) History of Poland<br>2. (0.7852) Alexander Jagiellon<br>3. (0.7636) Józefa Hennelowa<br>4. (0.7619) Kingdom of Poland (1385–1569)<br>5. (0.7586) Kutno<br>6. (0.7586) Pińczów<br>7. (0.7586) Pabianice<br>8. (0.7585) List of Jewish ghettos in G...<br>9. (0.7566) Skierniewice<br>10. (0.7537) Poland | 1. (0.8042) History of Poland<br>2. (0.7660) Poland<br>3. (0.7654) Skierniewice<br>4. (0.7629) Ostrołęka<br>5. (0.7626) Alexander Jagiellon<br>6. (0.7613) Kutno<br>7. (0.7613) Pabianice<br>8. (0.7613) Pińczów<br>9. (0.7544) Kingdom of Poland (1385–1569)<br>10. (0.7529) Giżycko | 1. (0.8794) Poland<br>2. (0.8780) History of Poland<br>3. (0.8739) Kingdom of Poland (1385–1569)<br>4. (0.8693) Lublin Voivodeship<br>5. (0.8666) Ostrołęka<br>6. (0.8651) Sierpc<br>7. (0.8620) Skierniewice<br>8. (0.8617) Greater Poland Voivodeship<br>9. (0.8612) Alexander Jagiellon<br>10. (0.8594) Trojany, Masovian Voivodeship |
| **bez IDF** | 1. (0.3676) Poland<br>2. (0.1468) History of Poland<br>3. (0.1430) Józef Piłsudski<br>4. (0.1262) Historiography<br>5. (0.1193) Soviet persecution of Poles...<br>6. (0.1083) Auschwitz concentration camp<br>7. (0.1012) Double genocide theory<br>8. (0.0990) Prussia<br>9. (0.0927) Białystok<br>10. (0.0918) Racism in Poland | 1. (0.6746) History of Poland<br>2. (0.6671) Micula (surname)<br>3. (0.6618) Skierniewice<br>4. (0.6576) Poland<br>5. (0.6558) Plovdiv<br>6. (0.6557) Slovenia<br>7. (0.6536) Săcele<br>8. (0.6516) Malbork<br>9. (0.6516) Kwidzyn<br>10. (0.6431) Volhynia | 1. (0.7555) History of Poland<br>2. (0.7294) Alexander Jagiellon<br>3. (0.7143) Kingdom of Poland (1385–1569)<br>4. (0.7135) Anna Komorowska<br>5. (0.7128) Józefa Hennelowa<br>6. (0.7074) Kutno<br>7. (0.7074) Pabianice<br>8. (0.7074) Pińczów<br>9. (0.7064) Skierniewice<br>10. (0.7039) Franciszek Ziejka | 1. (0.7569) History of Poland<br>2. (0.7137) Poland<br>3. (0.7103) Skierniewice<br>4. (0.7086) Alexander Jagiellon<br>5. (0.7066) Pabianice<br>6. (0.7066) Pińczów<br>7. (0.7066) Kutno<br>8. (0.7057) Ostrołęka<br>9. (0.7040) Kingdom of Poland (1385–1569)<br>10. (0.7003) Giżycko | 1. (0.8370) History of Poland<br>2. (0.8353) Poland<br>3. (0.8323) Kingdom of Poland (1385–1569)<br>4. (0.8241) Lublin Voivodeship<br>5. (0.8214) Ostrołęka<br>6. (0.8211) Sierpc<br>7. (0.8194) Greater Poland Voivodeship<br>8. (0.8183) Skierniewice<br>9. (0.8173) Alexander Jagiellon<br>10. (0.8151) Trojany, Masovian Voivodeship |


## Zapytanie: 'climate change'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.2525) Intergovernmental Panel on ...<br>2. (0.1951) Global warming<br>3. (0.1214) Miocene<br>4. (0.1195) Ridhima Pandey<br>5. (0.1123) Xiomara Acevedo<br>6. (0.1075) Subarctic climate<br>7. (0.1014) Climate change<br>8. (0.1014) María Fernanda Espinosa<br>9. (0.0995) Woody plant encroachment<br>10. (0.0984) Climate sensitivity | 1. (0.7896) Utah<br>2. (0.7889) Frontenac, Gironde<br>3. (0.7774) Saline County, Nebraska<br>4. (0.7759) Regan, North Dakota<br>5. (0.7736) Taloga, Oklahoma<br>6. (0.7633) Montello, Wisconsin<br>7. (0.7596) Electoral district of Riverton<br>8. (0.7590) Saline County, Kansas<br>9. (0.7583) Poweshiek County, Iowa<br>10. (0.7577) Frontenac County | 1. (0.8185) Caquetá Department<br>2. (0.8157) Caazapá Department<br>3. (0.8145) Achaguas Municipality<br>4. (0.8105) Limón Province<br>5. (0.8084) El Llano, Dominican Republic<br>6. (0.8073) Chimborazo Province<br>7. (0.8069) Mocoa<br>8. (0.8065) Cauca Department<br>9. (0.8056) List of places in Guatemala<br>10. (0.8043) Colón Department | 1. (0.7258) Polar climate<br>2. (0.7236) Tropical rainforest climate<br>3. (0.6947) Oceanic climate<br>4. (0.6535) Ñuble Region<br>5. (0.6484) Climate of Nigeria<br>6. (0.6479) Arica<br>7. (0.6433) Itapúa<br>8. (0.6426) Paysandú Department<br>9. (0.6422) Salto Department<br>10. (0.6422) Artigas Department | 1. (0.6571) Polar climate<br>2. (0.6447) Penny Whetton<br>3. (0.6387) Climate crisis<br>4. (0.6312) Green Climate Fund<br>5. (0.6243) Climate of Nigeria<br>6. (0.6096) Climate change denial<br>7. (0.6075) Tropical rainforest climate<br>8. (0.6021) Intergovernmental Panel on ...<br>9. (0.6013) Oceanic climate<br>10. (0.5866) Rajendra K. Pachauri |
| **bez IDF** | 1. (0.2499) Intergovernmental Panel on ...<br>2. (0.1975) Global warming<br>3. (0.1152) Miocene<br>4. (0.1146) Ridhima Pandey<br>5. (0.1080) Xiomara Acevedo<br>6. (0.1001) Climate change<br>7. (0.1001) María Fernanda Espinosa<br>8. (0.0994) Woody plant encroachment<br>9. (0.0983) Subarctic climate<br>10. (0.0942) Marinel Sumook Ubaldo | 1. (0.7624) Taloga, Oklahoma<br>2. (0.7531) Utah<br>3. (0.7527) Frontenac, Gironde<br>4. (0.7428) Saline County, Nebraska<br>5. (0.7375) Regan, North Dakota<br>6. (0.7318) Frontenac County<br>7. (0.7308) Electoral district of Riverton<br>8. (0.7281) Järna, Vansbro Municipality<br>9. (0.7261) United States Senate Energy...<br>10. (0.7252) Marquette County, Michigan | 1. (0.7889) Achaguas Municipality<br>2. (0.7880) Caquetá Department<br>3. (0.7810) Mocoa<br>4. (0.7802) Cauca Department<br>5. (0.7796) El Llano, Dominican Republic<br>6. (0.7793) Caazapá Department<br>7. (0.7792) Bucaramanga<br>8. (0.7783) Guaviare River<br>9. (0.7766) Chimborazo Province<br>10. (0.7764) Azuay Province | 1. (0.6973) Tropical rainforest climate<br>2. (0.6891) Polar climate<br>3. (0.6573) Oceanic climate<br>4. (0.6150) Ñuble Region<br>5. (0.6147) Climate of Nigeria<br>6. (0.6138) Climate crisis<br>7. (0.6098) Arica<br>8. (0.6083) Penny Whetton<br>9. (0.6059) Itapúa<br>10. (0.6049) Alto Paraná Department | 1. (0.6445) Penny Whetton<br>2. (0.6362) Climate crisis<br>3. (0.6319) Green Climate Fund<br>4. (0.6260) Polar climate<br>5. (0.6100) Climate change denial<br>6. (0.6042) Intergovernmental Panel on ...<br>7. (0.6001) Climate of Nigeria<br>8. (0.5884) Rajendra K. Pachauri<br>9. (0.5871) Tropical rainforest climate<br>10. (0.5858) United Nations Framework Co... |


## Zapytanie: 'machine learning algorithms'

|     | bez SVD | SVD k=100 | SVD k=200 | SVD k=500 | SVD k=1000 |
| --- | --- | --- | --- | --- | --- |
| **z IDF** | 1. (0.4020) Genetic algorithm<br>2. (0.3411) Symmetric-key algorithm<br>3. (0.2598) Observational learning<br>4. (0.2480) Sewing machine<br>5. (0.1903) Machine learning<br>6. (0.1839) Algorithm<br>7. (0.1605) Holland's schema theorem<br>8. (0.1488) Unsupervised learning<br>9. (0.1404) Key generation<br>10. (0.1352) Data annotation | 1. (0.8704) Unsupervised learning<br>2. (0.8593) Teaching machine<br>3. (0.8581) Database normalisation<br>4. (0.8526) Coding theory<br>5. (0.8478) Transformer (machine learni...<br>6. (0.8421) Relational database<br>7. (0.8353) Optimization (disambiguation)<br>8. (0.8200) Model-based design<br>9. (0.8161) Substitution box<br>10. (0.8140) Modular Audio Recognition F... | 1. (0.7921) Teaching machine<br>2. (0.7477) Unsupervised learning<br>3. (0.7377) Richard M. Karp<br>4. (0.7359) Artificial intelligence<br>5. (0.7183) Optimization (disambiguation)<br>6. (0.7020) Computer engineering<br>7. (0.6948) Numerical control<br>8. (0.6909) Max Newman<br>9. (0.6875) Modular Audio Recognition F...<br>10. (0.6875) Electrical engineering | 1. (0.7008) Elias Howe<br>2. (0.6966) Sewing machine<br>3. (0.6855) Lexington, Virginia<br>4. (0.6845) Susan Willcox<br>5. (0.6843) Willcox, Arizona<br>6. (0.6832) Glasgow, Virginia<br>7. (0.6820) Rockbridge County, Virginia<br>8. (0.6728) Stitching awl<br>9. (0.6703) Raphine, Virginia<br>10. (0.6697) Cyrus McCormick | 1. (0.5493) Teaching machine<br>2. (0.5351) Turing machine<br>3. (0.5309) Turing complete<br>4. (0.5291) Machine learning<br>5. (0.5162) Artificial intelligence<br>6. (0.5109) Adversarial machine learning<br>7. (0.4988) Intelligent agent<br>8. (0.4910) Sewing machine<br>9. (0.4867) Elias Howe<br>10. (0.4855) Genetic algorithm |
| **bez IDF** | 1. (0.3309) Genetic algorithm<br>2. (0.3069) Sewing machine<br>3. (0.2896) Observational learning<br>4. (0.2747) Symmetric-key algorithm<br>5. (0.1844) Machine learning<br>6. (0.1576) The Turk<br>7. (0.1539) Unsupervised learning<br>8. (0.1496) Algorithm<br>9. (0.1293) Holland's schema theorem<br>10. (0.1291) Data annotation | 1. (0.8826) Unsupervised learning<br>2. (0.8585) Teaching machine<br>3. (0.8585) Database normalisation<br>4. (0.8510) Transformer (machine learni...<br>5. (0.8418) Coding theory<br>6. (0.8358) Relational database<br>7. (0.8302) Numerical control<br>8. (0.8280) Modular Audio Recognition F...<br>9. (0.8208) Generative model<br>10. (0.8203) JMP (statistical software) | 1. (0.7964) Teaching machine<br>2. (0.7346) Unsupervised learning<br>3. (0.7140) Artificial intelligence<br>4. (0.7044) Numerical control<br>5. (0.7039) Computer engineering<br>6. (0.7026) Richard M. Karp<br>7. (0.7009) Electrical engineering<br>8. (0.6895) SIMATIC<br>9. (0.6882) Max Newman<br>10. (0.6761) Atanasoff–Berry Computer | 1. (0.7645) Elias Howe<br>2. (0.7632) Sewing machine<br>3. (0.7521) Lexington, Virginia<br>4. (0.7515) Susan Willcox<br>5. (0.7513) Willcox, Arizona<br>6. (0.7502) Glasgow, Virginia<br>7. (0.7487) Rockbridge County, Virginia<br>8. (0.7401) Stitching awl<br>9. (0.7357) Raphine, Virginia<br>10. (0.7353) Cyrus McCormick | 1. (0.5818) Teaching machine<br>2. (0.5629) Sewing machine<br>3. (0.5629) Turing machine<br>4. (0.5533) Elias Howe<br>5. (0.5426) Glasgow, Virginia<br>6. (0.5421) Lexington, Virginia<br>7. (0.5410) Susan Willcox<br>8. (0.5395) Rockbridge County, Virginia<br>9. (0.5387) Willcox, Arizona<br>10. (0.5280) Machine |


### 4.4. Najważniejsze wnioski z eksperymentów

#### **Wpływ IDF**
W przeprowadzonych eksperymentach zaobserwowano, że wpływ zastosowania IDF jest bardziej subtelny niż można by oczekiwać. Możliwe przyczyny:
- **Wstępne filtrowanie słownika** - Poprzez eliminację stop-words oraz filtrowanie terminów przy tworzeniu słownika, terminy o potencjalnie najniższych wagach IDF mogły zostać usunięte jeszcze zanim IDF zostało zastosowane.
- **Charakter źródła dokumentów** - Artykuły z Simple English Wikipedia charakteryzują się ograniczonym i bardziej jednolitym słownictwem, co może wpływać na mniejsze zróżnicowanie wag IDF.

#### **Wpływ SVD**
- **Semantyczne powiązania** - Wyniki z SVD często zawierają dokumenty, które niekoniecznie zawierają dokładne terminy z zapytania, ale są semantycznie powiązane, co widać szczególnie w zapytaniach "quantum physics" i "climate change".
- **Zwiększenie poziomów dopasowania** - Zastosowanie SVD znacząco zwiększa poziom dopasowania wyników wyszukiwania - podobieństwo jest mierzone w przestrzeni konceptów, która jest bardziej zwarta od przestrzeni terminów.
- **Redukcja szumu** - W niektórych przypadkach SVD eliminuje z topowych wyników dokumenty, które mają przypadkowe dopasowania terminów, ale nie są semantycznie związane z tematem zapytania.

#### **Wpływ wartości k w SVD**
- **Niższe wartości k (100-200)** - Większa generalizacja, pojawiają się wyniki bardziej odległe semantycznie, często niezwiązane z wyszukiwaną frazą
- **Wyższe wartości k (500-1000)** - Z reguły lepsze (subiektywnie) dopasowanie wyników

## 5. Wnioski ogólne

Projekt demonstruje kompromisy między tradycyjnym podejściem TF-IDF a metodą Latent Semantic Indexing. Oba podejścia mają swoje zalety i wady:
- **Tradycyjne wyszukiwanie**: Szybsza inicjalizacja, dokładniejsze dopasowanie dla konkretnych terminów, ale ograniczona zdolność do znajdowania semantycznie powiązanych dokumentów.
- **Wyszukiwanie z SVD**: Potencjał do znajdowania semantycznie powiązanych dokumentów nawet przy braku dokładnego dopasowania terminów, kosztem większej złożoności obliczeniowej i potrzeby starannego doboru parametrów.

Projekt skutecznie łączy zaawansowane techniki algebry liniowej z praktycznym rozwiązaniem problemu wyszukiwania informacji, dostarczając funkcjonalny i wydajny system.